<a href="https://colab.research.google.com/github/nrohrbach/KeyFiguresElectricalInstallations/blob/main/KeyFiguresElectricalInstallations36kV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kennzahlen Geobasisdatensatz Elektrische Anlagen mit einer Nennspannung von über 36kV

---

Notebook um Kennzahlen des Geobasisdatensatzes "Elektrische Anlagen mit einer Nennspannung von über 36kV abzuleiten.


**Offene Fragen**

*   Sollen bei Leitungen die Erdungen auch gezählt werden oder nicht?
*   Hat Swissgrid auch Stationen der Netzebene 4?

**ToDo**

*   Kennzahlen Stationen nach Netzebene 2 und 4 unterteilen





In [ ]:
#pip install geopandas

In [ ]:
#pip install datetime

In [4]:
import geopandas as gpd
import pandas as pd
from datetime import datetime

In [5]:
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
dfLeitung = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')
dfStation = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Station')
dfMast = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Mast')

## Kennzahlen Leitungen

In [6]:
# Länge der Leitungen berechnen und nach Spannung aufsummieren
dfLeitung["Laenge"] = dfLeitung.length
dfLeitungKPI = dfLeitung.groupby(['StromnetzTyp','Spannung','LeitungTyp'])['Laenge'].sum()
dfLeitungKPI = dfLeitungKPI.to_frame().reset_index()
dfLeitungKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [ ]:
#Speichern
dfLeitungKPI.to_csv("Kennzahlen_Leitungen.csv", header=False, index=False, mode='a')

## Kennzahlen Masten

In [ ]:
dfMastKPI = dfMast.groupby(['StromnetzTyp','MastTyp'])['xtf_id'].count().reset_index()
dfMastKPI = dfMastKPI.rename(columns={'xtf_id':'Anzahl'})
dfMastKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [ ]:
#Speichern
dfMastKPI.to_csv("Kennzahlen_Masten.csv", header=True, index=False, mode='a')

## Kennzahlen Stationen

In [ ]:
dfStationKPI = dfStation.groupby(['StromnetzTyp','StationTyp'])['xtf_id'].count().reset_index()
dfStationKPI = dfStationKPI.rename(columns={'xtf_id':'Anzahl'})
dfStationKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [ ]:
#Speichern
dfStationKPI.to_csv("Kennzahlen_Stationen.csv", header=True, index=False, mode='a')

## Kennzahlen Werke

In [ ]:
# Leitungslänge pro Werk ableiten.
dfWerkeLeitung = dfLeitung
dfWerkeLeitung['Werk'] = dfWerkeLeitung['xtf_id'].str.split('_').str[0]
dfWerkeLeitung = dfWerkeLeitung.groupby(['Werk'])['Laenge'].sum()
dfWerkeLeitung = dfWerkeLeitung.to_frame().reset_index()
dfWerkeLeitung = dfWerkeLeitung.rename(columns={'Laenge':'LaengeLeitung'})

In [ ]:
# Anzahl Masten pro Werk ableiten.
dfWerkeMast = dfMast
dfWerkeMast['Werk'] = dfWerkeMast['xtf_id'].str.split('_').str[0]
dfWerkeMast = dfWerkeMast.groupby(['Werk'])['xtf_id'].count().reset_index()
dfWerkeMast = dfWerkeMast.rename(columns={'xtf_id':'AnzahlMasten'})

In [ ]:
# Anzahl Stationen pro Werk ableiten.
dfWerkeStation = dfStation
dfWerkeStation['Werk'] = dfWerkeStation['xtf_id'].str.split('_').str[0]
dfWerkeStation = dfWerkeStation.groupby(['Werk'])['xtf_id'].count().reset_index()
dfWerkeStation = dfWerkeStation.rename(columns={'xtf_id':'AnzahlStationen'})

In [ ]:
# Zusammenführen
dfWerke = pd.merge(dfWerkeLeitung, dfWerkeMast, how="left", on=["Werk"])
dfWerke = pd.merge(dfWerke, dfWerkeStation, how="left", on=["Werk"])
dfWerke = dfWerke.fillna(0)
dfWerke['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [ ]:
#Speichern
dfWerke.to_csv("Kennzahlen_Werke.csv", header=True, index=False, mode='a')

In [8]:
dfLeitungKPI

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum
0,Allgemeinstrom,Andere,Freileitung,3.785600e+06,2023-05-03
1,Allgemeinstrom,Andere,Kabelleitung,2.215100e+04,2023-05-03
2,Allgemeinstrom,S110kV,Freileitung,1.390137e+06,2023-05-03
3,Allgemeinstrom,S110kV,Kabelleitung,4.018931e+05,2023-05-03
4,Allgemeinstrom,S132kV,Kabelleitung,9.624992e+04,2023-05-03
5,Allgemeinstrom,S150kV,Freileitung,6.401418e+04,2023-05-03
6,Allgemeinstrom,S150kV,Kabelleitung,3.967614e+05,2023-05-03
7,Allgemeinstrom,S220kV,Freileitung,4.832284e+06,2023-05-03
8,Allgemeinstrom,S220kV,Kabelleitung,2.501321e+04,2023-05-03
9,Allgemeinstrom,S380kV,Freileitung,1.893143e+06,2023-05-03


In [91]:
df380 = dfLeitungKPI.loc[dfLeitungKPI['Spannung']=='S380kV']
df220 = dfLeitungKPI.loc[dfLeitungKPI['Spannung']=='S220kV']
df150 = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV']))]
df66 = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S66kV','S65kV','S60kV','S50kV','S36kV','Andere']))]

In [92]:
df66

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum
0,Allgemeinstrom,Andere,Freileitung,3.785600e+06,2023-05-03
1,Allgemeinstrom,Andere,Kabelleitung,2.215100e+04,2023-05-03
11,Allgemeinstrom,S50kV,Freileitung,3.623561e+05,2023-05-03
12,Allgemeinstrom,S50kV,Kabelleitung,6.831027e+05,2023-05-03
13,Allgemeinstrom,S60kV,Freileitung,1.045754e+04,2023-05-03
14,Allgemeinstrom,S60kV,Kabelleitung,3.599944e+04,2023-05-03
15,Allgemeinstrom,S65kV,Freileitung,1.428735e+04,2023-05-03
16,Allgemeinstrom,S65kV,Kabelleitung,1.326761e+03,2023-05-03


In [80]:
dfLeitungStorymap = pd.DataFrame()

In [81]:
dfLeitungStorymap['220kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S220kV') & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']
dfLeitungStorymap['380kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV') & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']


dfLeitungStorymap['220kV_Total'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S220kV')]['Laenge'].sum()
dfLeitungStorymap['380kV_Total'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV')]['Laenge'].sum()

dfLeitungStorymap

,220kV_Kabel,380kV_Kabel,220kV_Total,380kV_Total
8,25013.209003,NaN,4.857297e+06,1.915187e+06


In [60]:
dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV']))]['Laenge'].sum()

2349055.7257298105

In [29]:
dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV')]['Laenge'].sum()

1915187.111823888

In [ ]:
dfLeitungStorymap['150kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV'])) & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']